In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
#model.add(kr.layers.Dense(units=100, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit??
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
inputs[inputs>0] = 1
inputs[inputs<1] = 0 

In [7]:
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
#just for looking
print(inputs[0])
#print (inputs[0].size)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [9]:
##change later to test
model.fit(inputs, outputs, epochs=5, batch_size=150)

Epoch 1/5
60000/60000 [==============================] - 32s 525us/step - loss: 0.2137 - accuracy: 0.9338
Epoch 2/5
60000/60000 [==============================] - 34s 566us/step - loss: 0.0936 - accuracy: 0.9703
Epoch 3/5
60000/60000 [==============================] - 37s 623us/step - loss: 0.0604 - accuracy: 0.9801
Epoch 4/5
60000/60000 [==============================] - 36s 593us/step - loss: 0.0462 - accuracy: 0.9851
Epoch 5/5
60000/60000 [==============================] - 36s 602us/step - loss: 0.0381 - accuracy: 0.9879


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
type(testImages)

bytes

In [12]:
testImages[0:4]

b'\x00\x00\x08\x03'

In [13]:
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
#print(testImages[0])
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
testInputs[testInputs>0] = 1
testInputs[testInputs<1] = 0 
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

9724

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[1.60523622e-10, 2.90806601e-10, 2.84106905e-10, ...,
        9.99999881e-01, 4.79152482e-11, 3.16190345e-08],
       [3.47990538e-12, 3.69456039e-08, 1.00000000e+00, ...,
        2.33420350e-09, 2.73782993e-12, 1.59377431e-14],
       [2.09172857e-09, 9.99939203e-01, 1.58410717e-06, ...,
        2.68931599e-05, 2.65958697e-05, 4.82768669e-08],
       ...,
       [3.67554685e-12, 1.55712532e-09, 1.80588877e-11, ...,
        2.64769778e-06, 1.16468870e-08, 7.11255916e-06],
       [6.73805589e-09, 1.03370121e-08, 2.06534685e-11, ...,
        1.22291395e-08, 4.90985367e-05, 2.37469044e-10],
       [7.42410862e-07, 7.11848571e-08, 1.94238396e-07, ...,
        4.68236006e-10, 1.92289136e-07, 1.97598968e-08]], dtype=float32)

In [16]:
##Model complete

In [17]:
##Re-train with an extra layer

In [18]:
type(model)

keras.engine.sequential.Sequential

In [19]:
fineTune = kr.models.Sequential()
for layer in model.layers:
    fineTune.add(layer)
for layer in fineTune.layers:
    layer.trainable = False

In [20]:
fineTune.add(kr.layers.Dense(units=10, activation='sigmoid'))

In [21]:
fineTune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fineTune.fit(inputs, outputs, epochs=5, batch_size=50)

Epoch 1/10
60000/60000 [==============================] - 19s 315us/step - loss: 0.3991 - accuracy: 0.8862
Epoch 2/10
60000/60000 [==============================] - 18s 294us/step - loss: 0.1560 - accuracy: 0.9850
Epoch 3/10
60000/60000 [==============================] - 17s 288us/step - loss: 0.0826 - accuracy: 0.9978
Epoch 4/10
60000/60000 [==============================] - 19s 315us/step - loss: 0.0489 - accuracy: 0.9980
Epoch 5/10
60000/60000 [==============================] - 19s 316us/step - loss: 0.0309 - accuracy: 0.9981
Epoch 6/10
60000/60000 [==============================] - 17s 277us/step - loss: 0.0208 - accuracy: 0.9982
Epoch 7/10
60000/60000 [==============================] - 17s 288us/step - loss: 0.0150 - accuracy: 0.9982
Epoch 8/10
60000/60000 [==============================] - 21s 348us/step - loss: 0.0116 - accuracy: 0.9983
Epoch 9/10
60000/60000 [==============================] - 39s 643us/step - loss: 0.0096 - accuracy: 0.9983
Epoch 10/10
60000/60000 [============

In [22]:
# Have the network predict the classes of the test inputs.
predictions = fineTune.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

9729

In [23]:
#Results
#save new model
fineTune.save('savedModel/freshlyTrained.h5')
#test in client